<a href="https://colab.research.google.com/github/kerenslendyrc/Estadistica/blob/main/Edpractica1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files # para cargar archivos en google colab
upload = files.upload()

Saving Base de datos.xlsx to Base de datos.xlsx


In [ ]:
!pip install factor-analyzer


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for factor-analyzer: filename=factor_analyzer-0.5.1-py2.py3-none-any.whl size=42655 sha256=ddc1ced13018e741cf5fdf9265783410b0d786d6bb66849147d9dafe82866bda
  Stored in directory: /root/.cache/pip/wheels/a2/af/06/f4d4ed4d9d714fda437fb1583629417319603c2266e7b233cc
Successfully built factor-analyzer


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN, OPTICS
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from factor_analyzer import FactorAnalyzer
from factor_analyzer.factor_analyzer import calculate_kmo, calculate_bartlett_sphericity



In [ ]:
import pandas as pd
import numpy as np
df = pd.read_excel("Base de datos.xlsx")

df_work = df.copy()

In [ ]:
df["fallecimiento_SN"] = df["fecha_muerte"].notna().map({True: "SI", False: "NO"})

In [ ]:
def capitulo_cie10(code):
    if pd.isna(code):
        return pd.NA
    c = str(code).strip()
    if len(c) == 0:
        return pd.NA
    return c[0]   # A, B, C, D, ...

df["capitulo_causa_muerte"] = df["cod_causa_basica_muerte"].apply(capitulo_cie10)


In [ ]:
map_capitulos = {
    "A": "Infecciosas_parasitarias",
    "B": "Infecciosas_parasitarias",
    "C": "Neoplasias",
    "D": "Neoplasias_sangre",
    "E": "Endocrinas_nutricionales",
    "F": "Trastornos_mentales",
    "G": "Sistema_nervioso",
    "H": "Ojo_oido",
    "I": "Cardiovasculares",
    "J": "Respiratorias",
    "K": "Digestivas",
    "L": "Piel_tejido_subcutaneo",
    "M": "Musculoesqueleticas",
    "N": "Genitourinarias",
    "O": "Embarazo_parto_puerperio",
    "P": "Perinatales",
    "Q": "Malformaciones_congenitas",
    "R": "Sintomas_signos_mal_definidos",
    "S": "Traumatismos",
    "T": "Envenenamientos_y_otros_consec",
    "V": "Causas_externas_transporte",
    "W": "Causas_externas_otros",
    "X": "Causas_externas_otros2",
    "Y": "Causas_externas_otros3",
    "Z": "Factores_contacto_servicios",
}

# Rehacer la columna agrupada:
df["capitulo_causa_muerte"] = df["cod_causa_basica_muerte"].apply(capitulo_cie10)
df["grupo_causa_muerte"] = df["capitulo_causa_muerte"].map(map_capitulos)

# Si alguna letra no estaba en el diccionario, la mandas a "OTRO"
df["grupo_causa_muerte"] = df["grupo_causa_muerte"].fillna("OTRO")

# Ver ahora las categorías
df["grupo_causa_muerte"].value_counts()


,count
grupo_causa_muerte,
OTRO,138271
Neoplasias,682
Sintomas_signos_mal_definidos,615
Respiratorias,503
Cardiovasculares,346
Infecciosas_parasitarias,143
Digestivas,119
Neoplasias_sangre,90
Genitourinarias,74


In [ ]:
# 1) Crear fallecimiento_SN si no lo tienes aún
df["fallecimiento_SN"] = df["fecha_muerte"].notna().map({True: "SI", False: "NO"})

# 2) Crear capítulo y grupo solo para quienes tienen código
df["capitulo_causa_muerte"] = df["cod_causa_basica_muerte"].apply(capitulo_cie10)

df["grupo_causa_muerte"] = df["capitulo_causa_muerte"].map(map_capitulos)

# 3) Diferenciar bien:
# - NO_APLICA si no falleció
# - OTRO solo si falleció pero la letra no está en el map_capitulos
df.loc[df["fallecimiento_SN"] == "NO", "grupo_causa_muerte"] = "NO_APLICA"
df.loc[
    (df["fallecimiento_SN"] == "SI") & (df["grupo_causa_muerte"].isna()),
    "grupo_causa_muerte"
] = "OTRO"

df["grupo_causa_muerte"].value_counts()


,count
grupo_causa_muerte,
NO_APLICA,138098
Neoplasias,681
Sintomas_signos_mal_definidos,615
Respiratorias,502
Cardiovasculares,346
OTRO,175
Infecciosas_parasitarias,143
Digestivas,119
Neoplasias_sangre,90


In [ ]:
def agrupar_uo_medico_macro(uo):
    if pd.isna(uo):
        return "OTRA"
    u = str(uo).upper()

    # Cuidados intensivos
    if "INTEN" in u:  # 1MHINTEN
        return "UCI"

    # Pediatría
    if "PEDIA" in u or "CXPED" in u or "ORTPE" in u or "NEONA" in u or "ORPED" in u or "OFPED" in u:
        return "PEDIATRIA"

    # Ginecología / Obstetricia
    if "GINEC" in u or "GINEO" in u or "CACUE" in u:  # CACUE lo mando a gineco-onco
        return "GINECOLOGIA"

    # Medicina Interna
    if "MEINT" in u:
        return "MEDICINA_INTERNA"

    # Cirugía (general y subespecialidades CX***, COLOP, etc.)
    if "CX" in u or "COLOP" in u:
        return "CIRUGIA"

    # Urología
    if "UROLO" in u or "UROLP" in u or "URONC" in u:
        return "UROLOGIA"

    # Hematología / Hemato-oncología
    if "HEMAT" in u or "HEMTR" in u or "HEONP" in u or "HE-ON" in u or "HEMAP" in u:
        return "HEMATOLOGIA_ONCO"

    # Oncología (códigos ONCO específicos)
    if "ONCOL" in u or "ONCOP" in u or "CXONC" in u or "ORON" in u:
        return "ONCOLOGIA"

    # Oftalmología
    if "OFTAL" in u or "OFPLA" in u or "RETIN" in u or "OFPED" in u or "NEUOF" in u:
        return "OFTALMOLOGIA"

    # Neurología
    if "NEURO" in u or "NEURP" in u:
        return "NEUROLOGIA"

    # Nefrología
    if "NEFRO" in u or "NEFTR" in u or "NEFRP" in u:
        return "NEFROLOGIA"

    # Ortopedia
    if "ORTOP" in u or "ORTOD" in u:
        return "ORTOPEDIA"

    # Alergia
    if "ALERG" in u:
        return "ALERGOLOGIA"

    # Geriatría
    if "GERIA" in u:
        return "GERIATRIA"

    # Cardiología
    if "CARDI" in u:
        return "CARDIOLOGIA"

    # Gastroenterología
    if "GASTR" in u or "GASTP" in u:
        return "GASTROENTEROLOGIA"

    # Otorrino
    if "OTORR" in u or "OTOLO" in u:
        return "OTORRINOLARINGOLOGIA"

    # Imagen/diagnóstico (radiología, med nuclear, laboratorio especial)
    if "RADIN" in u or "RADIO" in u or "MENUC" in u or "MELAB" in u:
        return "IMAGENOLOGIA"

    # Reumatología
    if "REUMA" in u or "REUMP" in u:
        return "REUMATOLOGIA"

    # Endocrinología
    if "ENDOC" in u:
        return "ENDOCRINOLOGIA"

    # Urgencias
    if "URGEN" in u:
        return "URGENCIAS"

    # Anestesiología
    if "ANECL" in u:
        return "ANESTESIOLOGIA"

    return "OTRA"

# Recalcular la macro-categoría
df["uo_medico_macro"] = df["uo_medico"].apply(agrupar_uo_medico_macro)

# Ver cómo queda ahora la distribución
df["uo_medico_macro"].value_counts()


,count
uo_medico_macro,
MEDICINA_INTERNA,54633
GINECOLOGIA,30799
PEDIATRIA,17409
CIRUGIA,15668
HEMATOLOGIA_ONCO,6746
UROLOGIA,4853
ORTOPEDIA,3865
ANESTESIOLOGIA,2861
URGENCIAS,1205


In [ ]:
df.loc[df["uo_medico_macro"] == "OTRA", "uo_medico"].value_counts().head(20)


,count
uo_medico,
1MHPERIN,6
1MHINFEC,5
1MHANDRO,4
1MHCARDP,3
1MHCARIN,3
1MHINTEP,2
1MHOFCOR,2
1MHOFSEG,2
1MHNEUMO,1


In [ ]:
# Ver las unidades más frecuentes
vc_uo = df["id_discapacidad"].value_counts()
top_uo = vc_uo.head(50).index   # por ejemplo, las 20 más frecuentes

# Crear grupo: top unidades vs resto
df["id_discapacidad_top"] = df["id_discapacidad"].where(df["id_discapacidad"].isin(top_uo), "OTRAS_UNIDADES")

df["id_discapacidad_top"].value_counts()


,count
id_discapacidad_top,
Ninguna,103486
OTRAS_UNIDADES,27745
Movimiento cuerpo extremidades,3277
El sistema nervioso,1457
Cardiorespiratorio y/o defensas,1278
"El sistema nervioso, Movimiento cuerpo extremidades",831
Los ojos,668
Los oÃ­dos,308
RENAL,298


In [ ]:
def agrupar_discapacidad_macro(desc):
    # 1) Faltantes o punto suelto
    if pd.isna(desc):
        return "DESCONOCIDA"
    s = str(desc).upper().strip()

    # 2) Sin discapacidad
    if "NINGUNA" in s or s == "NINGUNA" or "NO APLICA" in s:
        return "SIN_DISCAPACIDAD"

    # 3) Preparamos flags por sistema / tipo
    grupos = []

    # Motora / aparato locomotor
    if "MOVIMIENTO CUERPO" in s or "EXTREMIDADES" in s:
        grupos.append("MOTORA")
    if "ÓSEA" in s or "Ã“SEA" in s or "OSEA" in s:
        grupos.append("MOTORA")

    # Neurológica / cognitiva
    if "SISTEMA NERVIOSO" in s or "DEMENCIA" in s:
        grupos.append("NEUROLOGICA")

    # Cardiorrespiratorio y defensas
    if "CARDIORRESPIRATORIO" in s or "CARDIORESPIRATORIO" in s or "DEFENSAS" in s:
        grupos.append("CARDIORESPIRATORIA_DEFENSAS")

    # Sensorial visual
    if "LOS OJOS" in s or "OJOS" in s:
        grupos.append("SENSORIAL_VISUAL")

    # Sensorial auditiva (ojo con caracteres raros)
    if "LOS OÍDOS" in s or "LOS OIDOS" in s or "LOS OÃDOS" in s or "OÍDOS" in s or "OIDOS" in s or "OÃDOS" in s:
        grupos.append("SENSORIAL_AUDITIVA")

    # Comunicación (voz y habla)
    if "VOZ" in s or "HABLA" in s:
        grupos.append("COMUNICACION")

    # Renal
    if "RENAL" in s:
        grupos.append("RENAL")

    # Metabólica / endocrina / digestiva
    if "DIGESTI" in s or "METABOL" in s or "HORMONAL" in s or "OBESIDAD" in s or "ENDOCR" in s:
        grupos.append("METABOLICA_ENDOCRINA")

    # Piel
    if "LA PIEL" in s or "PIEL" in s:
        grupos.append("PIEL")

    # Genital / reproductivo
    if "GENITAL" in s or "REPRODUCTIVO" in s:
        grupos.append("GENITAL_REPRODUCTIVO")

    # Inmunidad / hematológica / onco-hemato
    if "INMUNIDAD" in s or "HEMATOLOGICA" in s or "CITOPENIAS" in s or "HODGKIN" in s:
        grupos.append("INMUNO_HEMATOLOGICA")

    # 4) Si tiene varias áreas a la vez → MULTIPLE
    grupos_unicos = list(set(grupos))

    if len(grupos_unicos) == 0:
        # Casos especiales que son más bien diagnósticos pero los tiramos a INMUNO/HEMATO o OTRA
        if "CA DE MAMA" in s or "MAMA" in s:
            return "INMUNO_HEMATOLOGICA"
        if "QUIMIOTERAPIA" in s:
            return "INMUNO_HEMATOLOGICA"
        if "EDAD" in s:
            return "MOTORA"   # fragilidad asociada a edad, si quieres puedes crear una categoría aparte
        if "OTRAS_UNIDADES" in s or s == ".":
            return "OTRA"
        # Nada matchea → OTRA
        return "OTRA"

    if len(grupos_unicos) == 1:
        return grupos_unicos[0]

    # Si tiene más de un sistema comprometido
    return "MULTIPLE"


# Aplicar sobre la columna de texto de discapacidad
df["tipo_discapacidad_macro"] = df["id_discapacidad_top"].apply(agrupar_discapacidad_macro)

# Ver cómo quedó
df["tipo_discapacidad_macro"].value_counts()


,count
tipo_discapacidad_macro,
SIN_DISCAPACIDAD,103591
OTRA,28118
MOTORA,3443
MULTIPLE,1585
NEUROLOGICA,1476
CARDIORESPIRATORIA_DEFENSAS,1278
SENSORIAL_VISUAL,704
RENAL,332
METABOLICA_ENDOCRINA,232


In [ ]:
df.loc[df["tipo_discapacidad_macro"] == "OTRA", "id_discapacidad_top"] \
  .value_counts() \
  .head(30)


,count
id_discapacidad_top,
OTRAS_UNIDADES,27745
Los oÃ­dos,308
.,41
Olfato tacto y/o gusto,24


In [ ]:
# 1) Los oídos → sensorial auditiva
mask_oidos = df["id_discapacidad_top"].str.contains("Los oÃ­dos", case=False, na=False)
df.loc[mask_oidos, "tipo_discapacidad_macro"] = "SENSORIAL_AUDITIVA"

# 2) Olfato tacto y/o gusto → sensorial otros
mask_olfato = df["id_discapacidad_top"].str.contains("Olfato tacto y/o gusto", case=False, na=False)
df.loc[mask_olfato, "tipo_discapacidad_macro"] = "SENSORIAL_OTROS"

# 3) Si quieres separar el punto como DESCONOCIDA
mask_punto = df["id_discapacidad_top"].eq(".")
df.loc[mask_punto, "tipo_discapacidad_macro"] = "DESCONOCIDA"

# Ver cómo queda
df["tipo_discapacidad_macro"].value_counts()


,count
tipo_discapacidad_macro,
SIN_DISCAPACIDAD,103533
OTRA,27745
MOTORA,3348
MULTIPLE,1585
NEUROLOGICA,1476
CARDIORESPIRATORIA_DEFENSAS,1278
SENSORIAL_VISUAL,668
SENSORIAL_AUDITIVA,522
RENAL,332


In [ ]:
import numpy as np
import pandas as pd

# 1) Estandarizar nombres de ciudad y departamento
df['ciudad_clean'] = (
    df['ciudad']
    .astype(str)
    .str.strip()
    .str.upper()
)

df['departamento_clean'] = (
    df['departamento']
    .astype(str)
    .str.strip()
    .str.upper()
)

# 2) Diccionario municipio -> provincia (solo SANTANDER)
provincia_map = {
    # --- METROPOLITANA ---
    'BUCARAMANGA': 'Metropolitana',
    'FLORIDABLANCA': 'Metropolitana',
    'GIRÓN': 'Metropolitana',
    'GIRON': 'Metropolitana',        # por si viene sin tilde
    'PIEDECUESTA': 'Metropolitana',
    'LEBRIJA': 'Metropolitana',
    'LOS SANTOS': 'Metropolitana',
    'RIONEGRO': 'Metropolitana',
    'SANTA BÁRBARA': 'Metropolitana',
    'SANTA BARBARA': 'Metropolitana',
    'ZAPATOCA': 'Metropolitana',
    'EL PLAYÓN': 'Metropolitana',
    'EL PLAYON': 'Metropolitana',
    # Decide tú si TONA va aquí o en Soto Norte:
    'TONA': 'Metropolitana',

    # --- COMUNERA ---
    'CHIMA': 'Comunera',
    'CONFINES': 'Comunera',
    'CONTRATACIÓN': 'Comunera',
    'CONTRATACION': 'Comunera',
    'EL GUACAMAYO': 'Comunera',
    'GALÁN': 'Comunera',
    'GALAN': 'Comunera',
    'GÁMBITA': 'Comunera',
    'GAMBITA': 'Comunera',
    'GUADALUPE': 'Comunera',
    'GUAPOTÁ': 'Comunera',
    'GUAPOTA': 'Comunera',
    'HATO': 'Comunera',
    'OIBA': 'Comunera',
    'PALMAR': 'Comunera',
    'PALMAS DEL SOCORRO': 'Comunera',
    'SANTA HELENA DEL OPÓN': 'Comunera',
    'SANTA HELENA DEL OPON': 'Comunera',
    'SIMACOTA': 'Comunera',
    'SOCORRO': 'Comunera',
    'SUAITA': 'Comunera',

    # --- VÉLEZ ---
    'AGUADA': 'Vélez',
    'ALBANIA': 'Vélez',
    'BARBOSA': 'Vélez',
    'BOLÍVAR': 'Vélez',
    'BOLIVAR': 'Vélez',
    'CHIPATÁ': 'Vélez',
    'CHIPATA': 'Vélez',
    'CIMITARRA': 'Vélez',
    'EL PEÑÓN': 'Vélez',
    'EL PENON': 'Vélez',
    'FLORIÁN': 'Vélez',
    'FLORIAN': 'Vélez',
    'GUAVATÁ': 'Vélez',
    'GUAVATA': 'Vélez',
    'GÜEPSA': 'Vélez',
    'GUEPSA': 'Vélez',
    'JESÚS MARÍA': 'Vélez',
    'JESUS MARIA': 'Vélez',
    'LA BELLEZA': 'Vélez',
    'LA PAZ': 'Vélez',
    'LANDÁZURI': 'Vélez',
    'LANDAZURI': 'Vélez',
    'PUENTE NACIONAL': 'Vélez',
    'PUERTO PARRA': 'Vélez',
    'SAN BENITO': 'Vélez',
    'SUAREZ': 'Vélez',   # si apareciera así en tus datos
    'SUCRE': 'Vélez',
    'VÉLEZ': 'Vélez',
    'VELEZ': 'Vélez',

    # --- GUANENTÁ ---
    'ARATOCA': 'Guanentá',
    'BARICHARA': 'Guanentá',
    'CABRERA': 'Guanentá',
    'CHARALÁ': 'Guanentá',
    'CHARALA': 'Guanentá',
    'COROMORO': 'Guanentá',
    'CURITÍ': 'Guanentá',
    'CURITI': 'Guanentá',
    'ENCINO': 'Guanentá',
    'JORDÁN': 'Guanentá',
    'JORDAN': 'Guanentá',
    'MOGOTES': 'Guanentá',
    'OCAMONTE': 'Guanentá',
    'ONZAGA': 'Guanentá',
    'PÁRAMO': 'Guanentá',
    'PARAMO': 'Guanentá',
    'PINCHOTE': 'Guanentá',
    'SAN GIL': 'Guanentá',
    'SAN JOAQUÍN': 'Guanentá',
    'SAN JOAQUIN': 'Guanentá',
    'VALLE DE SAN JOSÉ': 'Guanentá',
    'VALLE DE SAN JOSE': 'Guanentá',
    'VILLANUEVA': 'Guanentá',

    # --- GARCÍA ROVIRA ---
    'CAPITANEJO': 'García Rovira',
    'CARCASÍ': 'García Rovira',
    'CARCASI': 'García Rovira',
    'CEPITÁ': 'García Rovira',
    'CEPITA': 'García Rovira',
    'CERRITO': 'García Rovira',
    'CONCEPCIÓN': 'García Rovira',
    'CONCEPCION': 'García Rovira',
    'ENCISO': 'García Rovira',
    'GUACA': 'García Rovira',
    'MACARAVITA': 'García Rovira',
    'MÁLAGA': 'García Rovira',
    'MALAGA': 'García Rovira',
    'MOLAGAVITA': 'García Rovira',
    'SAN ANDRÉS': 'García Rovira',
    'SAN ANDRES': 'García Rovira',
    'SAN JOSÉ DE MIRANDA': 'García Rovira',
    'SAN JOSE DE MIRANDA': 'García Rovira',
    'SAN MIGUEL': 'García Rovira',

    # --- YARIGUÍES ---
    'BARRANCABERMEJA': 'Yariguíes',
    'BETULIA': 'Yariguíes',
    'EL CARMEN DE CHUCURÍ': 'Yariguíes',
    'EL CARMEN DE CHUCURI': 'Yariguíes',
    'PUERTO WILCHES': 'Yariguíes',
    'SABANA DE TORRES': 'Yariguíes',
    'SAN VICENTE DE CHUCURÍ': 'Yariguíes',
    'SAN VICENTE DE CHUCURI': 'Yariguíes',

    # --- SOTO / SOTO NORTE ---
    'CALIFORNIA': 'Soto Norte',
    'CHARTA': 'Soto Norte',
    'MATANZA': 'Soto Norte',
    'SURATÁ': 'Soto Norte',
    'SURATA': 'Soto Norte',
    'VETAS': 'Soto Norte',
    # Si prefieres que Tona quede aquí:
    # 'TONA': 'Soto Norte',
}

# 3) Crear la variable provincia_ciudad para SANTANDER
df['provincia_ciudad'] = df['ciudad_clean'].map(provincia_map)

# 4) Asignar 'Otros departamentos' para todo lo que no es Santander
df.loc[df['departamento_clean'] != 'SANTANDER', 'provincia_ciudad'] = 'Otros departamentos'

# 5) Cualquier municipio de Santander no mapeado explícitamente también va a 'Otros departamentos'
df['provincia_ciudad'] = df['provincia_ciudad'].fillna('Otros departamentos')

# Opcional: revisar distribución
print(df['provincia_ciudad'].value_counts(dropna=False))


provincia_ciudad
Metropolitana          116917
Otros departamentos      9042
Yariguíes                6759
Guanentá                 3751
Vélez                    1832
Comunera                 1432
García Rovira             852
Soto Norte                570
Name: count, dtype: int64


In [ ]:
eps_counts = df['EPS'].value_counts(dropna=False)
print(eps_counts.head(30))


EPS
NUEVA EPS                        103886
FIDUPREVISORA                     12194
ECOPETROL                          4694
EPS SANITAS                        3975
COLSANITAS                         1884
EPS FAMISANAR                      1728
SURAMERICANA SA                    1693
CLINICA SANTA CRUZ DE LA LOMA      1446
COOSALUD EPS                       1269
AXA COLPATRIA                      1216
SALUD TOTAL EPS                     795
ALLIANZ SEGUROS                     720
MEDISANITAS                         623
COLMEDICA MEDICINA PREPAGADA        537
COOMEVA EPS                         439
UIS                                 397
SEGUROS BOLIVAR                     360
PACIENTE PARTICULAR                 288
SALUD TOTAL SUBSIADO                250
HDI SEGUROS (ANTES LIBERTY)         224
MED PLUS MEDICINA PREPAGADA         212
COMPENSAR EPS                       198
CORSO SALAMANCA JAIRO               183
FUNDACION MEDICO PREVENTIVA         169
COMPAÃ‘IA MUNDIAL DE SEGUROS        

In [ ]:
# 1) Limpieza básica
df['EPS_clean'] = (
    df['EPS']
    .astype(str)
    .str.strip()
    .str.upper()
)

# 2) Unificar TODAS las variantes que contengan "NUEVA EPS"
mask_nueva = df['EPS_clean'].str.contains('NUEVA EPS', na=False)
df.loc[mask_nueva, 'EPS_clean'] = 'NUEVA EPS'

# 3) Recalcular frecuencias sobre la columna ya unificada
eps_freq = df['EPS_clean'].value_counts(normalize=True)

umbral_grande = 0.05
umbral_mediana_inf = 0.01

def clasificar_eps(eps):
    if pd.isna(eps):
        return 'EPS_no_reportada'
    if eps == 'NUEVA EPS':
        return 'EPS_NUEVA'
    f = eps_freq.get(eps, 0)
    if f >= umbral_grande:
        return 'EPS_grandes'
    elif f >= umbral_mediana_inf:
        return 'EPS_medianas'
    else:
        return 'EPS_pequeñas'

df['EPS_grupo'] = df['EPS_clean'].apply(clasificar_eps)

print(df.loc[df['EPS_clean'] == 'NUEVA EPS', 'EPS_grupo'].value_counts())


EPS_grupo
EPS_NUEVA    103887
Name: count, dtype: int64


In [ ]:
# Ver todas las variantes que contienen "NUEVA EPS"
mask_nueva = df['EPS_clean'].str.contains('NUEVA EPS', na=False)

print(df.loc[mask_nueva, 'EPS_clean'].value_counts())
print(df.loc[mask_nueva, 'EPS_grupo'].value_counts())


EPS_clean
NUEVA EPS    103887
Name: count, dtype: int64
EPS_grupo
EPS_NUEVA    103887
Name: count, dtype: int64


In [ ]:
import pandas as pd

# Tabla con EPS limpia, grupo y frecuencia absoluta
tabla_eps = (
    df
    .groupby(['EPS_grupo', 'EPS_clean'])
    .size()
    .reset_index(name='n_pacientes')
    .sort_values(['EPS_grupo', 'n_pacientes'], ascending=[True, False])
)

print("=== EPS por grupo (con número de pacientes) ===")
print(tabla_eps)

# Si quieres ver solo los nombres de EPS por grupo:
eps_por_grupo = (
    tabla_eps
    .groupby('EPS_grupo')['EPS_clean']
    .apply(list)
)

print("\n=== Listado de EPS en cada grupo ===")
for grupo, lista_eps in eps_por_grupo.items():
    print(f"\nGrupo: {grupo}")
    for eps in lista_eps:
        print(f"  - {eps}")


=== EPS por grupo (con número de pacientes) ===
       EPS_grupo                   EPS_clean  n_pacientes
0      EPS_NUEVA                   NUEVA EPS       103887
1    EPS_grandes               FIDUPREVISORA        12194
4   EPS_medianas                   ECOPETROL         4694
6   EPS_medianas                 EPS SANITAS         3975
3   EPS_medianas                  COLSANITAS         1884
..           ...                         ...          ...
53  EPS_pequeñas  LA Â CLINIC Â DU CORPS LTD            1
78  EPS_pequeñas   TRAVEL GUARD AMERICAS LLC            1
80  EPS_pequeñas                      UNACSA            1
83  EPS_pequeñas    UNION TEMPORAL TOLIHUILA            1
86  EPS_pequeñas        YEPES PEREZ HERNANDO            1

[87 rows x 3 columns]

=== Listado de EPS en cada grupo ===

Grupo: EPS_NUEVA
  - NUEVA EPS

Grupo: EPS_grandes
  - FIDUPREVISORA

Grupo: EPS_medianas
  - ECOPETROL
  - EPS SANITAS
  - COLSANITAS
  - EPS FAMISANAR
  - SURAMERICANA SA
  - CLINICA SANTA CRU

dummies


In [ ]:
# 2) Columnas categóricas que sí vamos a convertir en dummies
cols_a_dummificar = [
    "provincia_ciudad",
    "Servicio",
    "asignacion_enfermeria",
    "gran_causa_morbilidad",
    "uo_medico_macro",
    "sexo",
    "EPS_grupo",
    "tipo_discapacidad_macro",
    "fallecimiento_SN"   # SÍ / NO
]

# Por seguridad: nos quedamos solo con las que realmente existan en df
cols_existentes = [c for c in cols_a_dummificar if c in df.columns]
cols_faltantes = [c for c in cols_a_dummificar if c not in df.columns]

print("Dummificando columnas:", cols_existentes)
print("No se encontraron en df:", cols_faltantes)


Dummificando columnas: ['provincia_ciudad', 'Servicio', 'asignacion_enfermeria', 'gran_causa_morbilidad', 'uo_medico_macro', 'sexo', 'EPS_grupo', 'tipo_discapacidad_macro', 'fallecimiento_SN']
No se encontraron en df: []


In [ ]:
cols_cardinalidad = [
    "provincia_ciudad",
    "Servicio",
    "asignacion_enfermeria",
    "gran_causa_morbilidad",
    "uo_medico_macro",
    "sexo",
    "EPS_grupo",
    "tipo_discapacidad_macro",
    "fallecimiento_SN"
]

print("=== Cardinalidad de variables categóricas ===\n")

resumen = []
for col in cols_cardinalidad:
    n_cat = df[col].nunique(dropna=True)
    resumen.append({"variable": col, "n_categorias": n_cat})

    print(f"Variable: {col}")
    print(f" - Número de categorías: {n_cat}")
    print(" - Top 10 categorías más frecuentes:")
    print(df[col].value_counts(dropna=False).head(10))
    print("\n" + "-"*60 + "\n")


=== Cardinalidad de variables categóricas ===

Variable: provincia_ciudad
 - Número de categorías: 8
 - Top 10 categorías más frecuentes:
provincia_ciudad
Metropolitana          116917
Otros departamentos      9042
Yariguíes                6759
Guanentá                 3751
Vélez                    1832
Comunera                 1432
García Rovira             852
Soto Norte                570
Name: count, dtype: int64

------------------------------------------------------------

Variable: Servicio
 - Número de categorías: 5
 - Top 10 categorías más frecuentes:
Servicio
Hospitalización adultos       95766
Ginecobstetricia              22748
Hospitalización pediatrica    18748
UCI Adultos                    2980
UCI Pediatrica                  913
Name: count, dtype: int64

------------------------------------------------------------

Variable: asignacion_enfermeria
 - Número de categorías: 28
 - Top 10 categorías más frecuentes:
asignacion_enfermeria
1 UE HOSP URGENCIAS CAL P5        42

In [ ]:
# 3) Dummies 0/1 para las columnas categóricas
df_dummies = pd.get_dummies(
    df[cols_existentes],
    columns=cols_existentes,
    drop_first=False,   # conserva todas las categorías (útil para análisis descriptivo)
    dtype=int
)

# 4) Construir un DataFrame final mezclando edad + dummies
X_dummis = pd.concat(
    [
        df[["edad"]].reset_index(drop=True),  # edad numérica
        df_dummies.reset_index(drop=True)
    ],
    axis=1
)

X_dummis.head()


,edad,provincia_ciudad_Comunera,provincia_ciudad_García Rovira,provincia_ciudad_Guanentá,provincia_ciudad_Metropolitana,provincia_ciudad_Otros departamentos,provincia_ciudad_Soto Norte,provincia_ciudad_Vélez,provincia_ciudad_Yariguíes,Servicio_Ginecobstetricia,...,tipo_discapacidad_macro_NEUROLOGICA,tipo_discapacidad_macro_OTRA,tipo_discapacidad_macro_PIEL,tipo_discapacidad_macro_RENAL,tipo_discapacidad_macro_SENSORIAL_AUDITIVA,tipo_discapacidad_macro_SENSORIAL_OTROS,tipo_discapacidad_macro_SENSORIAL_VISUAL,tipo_discapacidad_macro_SIN_DISCAPACIDAD,fallecimiento_SN_NO,fallecimiento_SN_SI
0,31,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
1,77,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
2,76,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
3,67,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
4,67,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0


In [ ]:
# 1) Matriz de entrada para PCA
X = X_dummis.copy()
features = X.columns

# 2) Estandarización (media 0, varianza 1)
scaler = StandardScaler(with_mean=True, with_std=True)
X_scaled = scaler.fit_transform(X)


In [ ]:
from sklearn.decomposition import PCA
import numpy as np

# X_scaled es tu matriz estandarizada (la que ya tenías)
n_muestras = min(30000, X_scaled.shape[0])  # por ejemplo, máximo 30k filas
idx = np.random.choice(X_scaled.shape[0], size=n_muestras, replace=False)
X_sample = X_scaled[idx]

# PCA que elige automáticamente los componentes para explicar el 85% de la varianza
pca_sample = PCA(
    n_components=0.85,      # queremos ≥ 85% de varianza
    svd_solver="full",      # obligatorio para usar float como n_components
    random_state=0
)

X_pca_sample = pca_sample.fit_transform(X_sample)

# Número de componentes que alcanzan ≥85% en la muestra
n_components_85 = pca_sample.n_components_
print("Componentes necesarios (muestra) para ≥85% varianza:", n_components_85)

# Si quieres ver la varianza acumulada en la muestra:
explained_var_sample = pca_sample.explained_variance_ratio_
cum_var_sample = np.cumsum(explained_var_sample)
print("Varianza acumulada en la muestra:", cum_var_sample)



Componentes necesarios (muestra) para ≥85% varianza: 60
Varianza acumulada en la muestra: [0.05257005 0.0979808  0.13111598 0.15708116 0.18217004 0.20675347
 0.22661114 0.24584128 0.26400999 0.28216928 0.29979542 0.31704855
 0.33292677 0.34852579 0.36347293 0.3780879  0.39245808 0.40638151
 0.42000627 0.43305825 0.44606711 0.45882832 0.47140876 0.48336984
 0.49515716 0.50684314 0.51839764 0.52980607 0.54109483 0.5522205
 0.56320196 0.57409227 0.58485427 0.59554844 0.60617758 0.61676235
 0.62727261 0.63777141 0.6481938  0.65853771 0.6688651  0.67907132
 0.68924244 0.69931762 0.70936627 0.71937757 0.72919995 0.73892045
 0.74860556 0.75823543 0.7677572  0.77725279 0.78667171 0.79598828
 0.80525525 0.81449631 0.82366398 0.8327737  0.84178438 0.85072089]


In [ ]:
# Supón que tienes pca y explained_variance_ratio_
var_acum = pca_sample.explained_variance_ratio_.cumsum()
for k in [10, 20, 30, 40]:
    print(k, var_acum[k-1])


10 0.28216928175724804
20 0.43305824986689095
30 0.5522205021747885
40 0.6585377119230397


Analisis factorial

Inicialmente, con variables dummificadas sin agrupar, el PCA requería 263 componentes para el 85% de la varianza y el KMO era 0.055.

Tras agrupar ciudad en provincias / otros departamentos y agrupar EPS en categorías por tamaño, el número de componentes para 85% de la varianza se redujo a 61 y el KMO subió a 0.106.

Esto muestra una ligera mejora en la estructura común, pero los valores de KMO siguen indicando que el análisis factorial clásico no es adecuado para este conjunto de variables.

Por tanto, se utiliza el PCA principalmente como reducción de dimensión previa al clustering, y se consideran enfoques alternativos (p. ej. MCA / distancias de Gower) para tratar la naturaleza categórica de las variables.

In [ ]:
X = df_dummies.copy()

# --- KMO ---
kmo_all, kmo_model = calculate_kmo(X)
print(f"KMO global del modelo: {kmo_model:.3f}")

# --- Bartlett ---
chi_square_value, p_value = calculate_bartlett_sphericity(X)
print(f"Test de Bartlett: chi2 = {chi_square_value:.2f}, p-value = {p_value:.5f}")


/usr/local/lib/python3.12/dist-packages/factor_analyzer/utils.py:244: UserWarning: The inverse of the variance-covariance matrix was calculated using the Moore-Penrose generalized matrix inversion, due to its determinant being at or very close to zero.
  warnings.warn(


KMO global del modelo: 0.106
Test de Bartlett: chi2 = nan, p-value = nan


/usr/local/lib/python3.12/dist-packages/factor_analyzer/factor_analyzer.py:109: RuntimeWarning: invalid value encountered in log
  statistic = -np.log(corr_det) * (n - 1 - (2 * p + 5) / 6)


Se intentó ajustar un modelo de análisis factorial exploratorio sobre el conjunto completo de variables categóricas dummificadas, pero los indicadores de adecuación muestral (KMO) mostraron que el modelo factorial no resultaba apropiado, por lo que se descartó este enfoque global.

In [ ]:
from factor_analyzer.factor_analyzer import calculate_kmo

# X_dummies: tu matriz después de todas las recodificaciones
kmo_all, kmo_vars = calculate_kmo(X_dummis)

kmo_all  # KMO global
kmo_vars  # array con KMO por variable

# Ver variables con KMO más bajo
kmo_por_variable = pd.DataFrame({
    'variable': X_dummis.columns,
    'kmo': kmo_vars
}).sort_values('kmo')

kmo_por_variable.head(20)


,variable,kmo
0,edad,0.116233
1,provincia_ciudad_Comunera,0.116233
2,provincia_ciudad_García Rovira,0.116233
3,provincia_ciudad_Guanentá,0.116233
4,provincia_ciudad_Metropolitana,0.116233
5,provincia_ciudad_Otros departamentos,0.116233
6,provincia_ciudad_Soto Norte,0.116233
7,provincia_ciudad_Vélez,0.116233
8,provincia_ciudad_Yariguíes,0.116233
9,Servicio_Ginecobstetricia,0.116233


calculo de entropía:

In [ ]:
from sklearn.decomposition import PCA
import numpy as np

# Supongo que ya entrenaste el PCA:
# pca = PCA(...).fit(X)
# y que tu objeto se llama:
pca = pca_sample

# 1) Proporción de varianza explicada por cada componente
prop_var = pca.explained_variance_ratio_  # array de longitud n_components

# 2) Asegurar que sumen 1 (por si acaso)
p = prop_var / prop_var.sum()

# 3) Entropía (log natural)
entropy_pca = -np.sum(p * np.log(p))

# 4) (Opcional) Entropía normalizada entre 0 y 1
entropy_pca_norm = entropy_pca / np.log(len(p))

print("Entropía PCA:", entropy_pca)
print("Entropía PCA normalizada:", entropy_pca_norm)



Entropía PCA: 3.9776142011381332
Entropía PCA normalizada: 0.9714898540389051


In [ ]:
from factor_analyzer import FactorAnalyzer
import numpy as np

n_factors = 10  # ?????????????????

fa = FactorAnalyzer(n_factors=n_factors, rotation=None)
fa.fit(X)

var_factors, prop_var, cum_var = fa.get_factor_variance()

p = prop_var / prop_var.sum()

entropy_fa = -np.sum(p * np.log(p))
entropy_fa_norm = entropy_fa / np.log(len(p))

print("Entropía AF:", entropy_fa)
print("Entropía AF normalizada:", entropy_fa_norm)


/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Entropía AF: 2.2043573674150996
Entropía AF normalizada: 0.9573402408111568


Se calculó la entropía de Shannon asociada a la distribución de la varianza explicada por los componentes del PCA y los factores del AF.

Valores cercanos a 1 en la entropía normalizada indican que la varianza explicada se distribuye de manera relativamente homogénea entre un número elevado de componentes/factores, en lugar de concentrarse en unas pocas dimensiones dominantes.
Esto sugiere que la estructura latente de los datos es altamente dispersa y que ni el PCA ni el AF logran condensar la mayor parte de la información en un conjunto reducido de dimensiones.
Estas observaciones son coherentes con el bajo índice KMO obtenido, que ya advertía sobre una varianza común limitada entre las variables.”


PCA y AF no revelan pocos “factores fuertes”, sino muchos ejes débiles.

se usará el PCA principalmente como reducción técnica de dimensión antes del clustering, más que para interpretar factores/ componentes como constructos clínicos bien definidos.

K means